In [8]:
import numpy as np
import torch
import torch.nn.functional as F  #激活函数
from torch import nn
from torch import optim  #优化器
import torchvision
import matplotlib.pyplot as plt
import os

In [9]:
def get_data(root_dir):
  paths=[]
  labels=[]
  for root, dirs, files in os.walk(root_dir):
    for file in files:
      paths.append(os.path.join(root, file))
      index=file.find('_')
      labels.append(file[:index])
  return paths,labels

def get_folder(root_dir):
  folder_name=[]
  for entry in os.listdir(root_dir):
    full_path=os.path.join(root_dir, entry)
    if os.path.isdir(full_path):
      folder_name.append(entry)
  return folder_name

In [10]:
class CifarDataset(torch.utils.data.Dataset):
  def __init__(self, root_dir):
    #初始化数据集，包含一系列图片与对应标签
    super().__init__()
    #self.data_path=root_dir
    dataset=get_folder(root_dir)
    data, labels=get_data(root_dir)
    self.data=data
    self.label=labels
    if(dataset[0][-4:]=='test'):
      #test_labels=get_folder(dataset[0])
      test_data, test_labels=get_data(dataset[0])
      #train_labels=get_folder(dataset[1])
      train_data, train_labels=get_data(dataset[1])
    else:
      #test_labels=get_folder(dataset[1])
      test_data, test_labels=get_data(dataset[1])
      #train_labels=get_folder(dataset[0])
      train_data, train_labels=get_data(dataset[0])
    self.train_label=train_labels
    self.train=train_data
    self.test_label=test_labels
    self.test=test_data
    #raise NotImplementedError
  
  def __len__(self):
    #返回数据集大小
    return len(self.data)
    #raise NotImplementedError
  
  def __getitem__(self, index):
    #返回第index个样本
    return self.data[index], self.label[index]
    #raise NotImplementedError
  
  

In [14]:
train_dataset=CifarDataset('..\cifar10')
train_dataloader=torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
print(train_dataset[2])

('..\\cifar10\\cifar10_test\\airplane\\airplane_00002.png', 'airplane')
